# Octonion Multiplication

Examples from blog posts by [John D. Cook](https://www.johndcook.com/blog/services-2/)

## How far is xy from yx on average for quaternions? [5 July 2018](https://www.johndcook.com/blog/2018/07/05/quaternion-multiplication/)

In [1]:
import numpy as np
    
def random_unit_quaternion():
    x = np.random.normal(size=4)
    return x / np.linalg.norm(x)
    
def mult(x, y):
    return np.array([
        x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
        x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
        x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
        x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]
    ])
    
    
N = 10000
s = 0
for _ in range(N):
    x = random_unit_quaternion()
    y = random_unit_quaternion()
    s += np.linalg.norm(mult(x, y) - mult(y, x))
  
print(s/N)

1.1298974836513174


## Weakening the requirements of a group [8 July 2018](https://www.johndcook.com/blog/2018/07/08/weak-groups/)

## Python code for octonion and sedenion multiplication [9 July 2018](https://www.johndcook.com/blog/2018/07/09/octonioin-multiplication/)

In [2]:
import numpy as np

# quaternion multiplication
def qmult(x, y):
    return np.array([
        x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
        x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
        x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
        x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]
   ])

# quaternion conjugate
def qstar(x):
    return x*np.array([1, -1, -1, -1])

# octonion multiplication
def omult(x, y):
    # Split octonions into pairs of quaternions
    a, b = x[:4], x[4:]
    c, d = y[:4], y[4:]
   
    z = np.zeros(8)
    z[:4] = qmult(a, c) - qmult(qstar(d), b)
    z[4:] = qmult(d, a) + qmult(b, qstar(c))
    return z

In [3]:
from numpy.linalg import norm

def random_unit_octonion():
    x = np.random.normal(size=8)
    return x / norm(x)

x = random_unit_octonion()
y = random_unit_octonion()
z = random_unit_octonion()

In [4]:
print(x)
print(y)
print(z)

[-0.27081526  0.02633699  0.25724809 -0.00074405  0.42925884 -0.72749247
  0.28694677 -0.25286643]
[-0.4886542  -0.01375817 -0.62936828  0.27150134 -0.07340038  0.39324629
 -0.14854765  0.33032365]
[ 0.51745201 -0.29500003  0.24069908 -0.0691771  -0.0820579  -0.39132859
  0.6491547   0.03497861]


In [5]:
eps = 1e-15

# alternative identities

a = omult(omult(x, x), y)
b = omult(x, omult(x, y))
assert( norm(a - b) < eps )

a = omult(omult(x, y), y)
b = omult(x, omult(y, y))
assert( norm(a - b) < eps )

In [6]:
# Moufang identities
    
a = omult(z, omult(x, omult(z, y)))
b = omult(omult(omult(z, x), z), y)
assert( norm(a - b) < eps )

a = omult(x, omult(z, omult(y, z)))
b = omult(omult(omult(x, z), y), z)
assert( norm(a - b) < eps )

a = omult(omult(z,x), omult(y, z))
b = omult(omult(z, omult(x, y)), z)
assert( norm(a  - b) < eps )

a = omult(omult(z,x), omult(y, z))
b = omult(z, omult(omult(x, y), z))
assert( norm(a - b) < eps )

In [7]:
# norm condition
n = norm(omult(omult(x, y), z))
assert( abs(n - 1) < eps )

In [8]:
def ostar(x):
    mask = -np.ones(8)
    mask[0] = 1
    return x*mask
 
# sedenion multiplication
def smult(x, y):
    # Split sedenions into pairs of octonions
    a, b = x[:8], x[8:]
    c, d = y[:8], y[8:]
    z = np.zeros(16)
    z[:8] = omult(a, c) - omult(ostar(d), b)
    e[8:] = omult(d, a) + omult(b, ostar(c))
    return z

In [9]:
x

array([-0.27081526,  0.02633699,  0.25724809, -0.00074405,  0.42925884,
       -0.72749247,  0.28694677, -0.25286643])

In [10]:
ostar(x)

array([-0.27081526, -0.02633699, -0.25724809,  0.00074405, -0.42925884,
        0.72749247, -0.28694677,  0.25286643])

## Refactored code for division algebras [10 July 2018](https://www.johndcook.com/blog/2018/07/10/cayley-dickson/)

In [11]:
def conj(x):
        xstar = -x
        xstar[0] *= -1
        return xstar 

def CayleyDickson(x, y):
    n = len(x)

    if n == 1:
        return x*y

    m = n // 2

    a, b = x[:m], x[m:]
    c, d = y[:m], y[m:]
    z = np.zeros(n)
    z[:m] = CayleyDickson(a, c) - CayleyDickson(conj(d), b)
    z[m:] = CayleyDickson(d, a) + CayleyDickson(b, conj(c))
    return z

In [12]:
print(x)
print(conj(x))

[-0.27081526  0.02633699  0.25724809 -0.00074405  0.42925884 -0.72749247
  0.28694677 -0.25286643]
[-0.27081526 -0.02633699 -0.25724809  0.00074405 -0.42925884  0.72749247
 -0.28694677  0.25286643]


In [13]:
print(CayleyDickson(x, y))

[ 0.73854767  0.11841479 -0.14597569  0.04180779 -0.40101854  0.2493429
 -0.04071081 -0.43911657]


## Scratchwork

In [14]:
def nparr(arr):
    return np.array(arr)

In [15]:
q1 = nparr([2, 3, 1, 4])
q2 = nparr([3, -1, 2, -7])

print(f"{qmult(q1, q2) = }")
print(f"{CayleyDickson(q1, q2) = }")

qmult(q1, q2) = array([35, -8, 24,  5])
CayleyDickson(q1, q2) = array([35., -8., 24.,  5.])


In [16]:
# Complex multiplication (for testing)
def cmult(x, y):
    return np.array([x[0] * y[0] - x[1] * y[1],
                     x[0] * y[1] + x[1] * y[0]])

In [17]:
c1 = nparr([2, 3])
d1 = nparr([3, -1])

print(f"{cmult(c1, d1) = }")
print(f"{CayleyDickson(c1, d1) = }")

cmult(c1, d1) = array([9, 7])
CayleyDickson(c1, d1) = array([9., 7.])


In [18]:
# Three versions of Cayley-Dickson multiplication, based on versions found in the literature and online

def cayley_dickson(x, y, vers=1):
    """Cayley-Dickson multiplication

    Version 1: defined in [Schafer 1966], Encycl. of Math, & [Baez 2002]
    Version 2: defined in Wikipedia & [Schafer 1953], and implemented by J.D. Cook
    Version 3: No conjugation; no mu (just to see what happens here)

    NOTE: In several of references an additional quantity is included in the calculation
    (i.e., denoted by mu in Schafer 1966, by gamma in Schafer 1953, and by delta in Encycl
    of Math). Here, the versions have been coded in a form that makes this quantity
    essentially equal to -1.
    """
    n = len(x)

    if n == 1:
        return x * y

    m = n // 2

    a, b = x[:m], x[m:]
    c, d = y[:m], y[m:]
    
    z = np.zeros(n, dtype=int)  # Don't force conversion to floats

    if vers == 1:
        # (ac + db^, a^d + cb), where x^ denotes conjugate of x
        z[:m] = cayley_dickson(a, c, vers) - cayley_dickson(d, conj(b), vers)
        z[m:] = cayley_dickson(conj(a), d, vers) + cayley_dickson(c, b, vers)
    elif vers == 2:
        # (ac + d^b, da + bc^)
        z[:m] = cayley_dickson(a, c, vers) - cayley_dickson(conj(d), b, vers)
        z[m:] = cayley_dickson(d, a, vers) + cayley_dickson(b, conj(c), vers)
    elif vers == 3:
        # (ac + bd, ad + bc))
        z[:m] = cayley_dickson(a, c, vers) - cayley_dickson(b, d, vers)
        z[m:] = cayley_dickson(a, d, vers) + cayley_dickson(b, c, vers)
    else:
        raise ValueError(f"{vers} must be in [1, 2, 3]")
    return z

In [19]:
print(f"{cmult(c1, d1) = }")
print(f"{CayleyDickson(c1, d1) = }")
print(f"{cayley_dickson(c1, d1, 1) = }")
print(f"{cayley_dickson(c1, d1, 2) = }")
print(f"{cayley_dickson(c1, d1, 3) = }")

cmult(c1, d1) = array([9, 7])
CayleyDickson(c1, d1) = array([9., 7.])
cayley_dickson(c1, d1, 1) = array([9, 7])
cayley_dickson(c1, d1, 2) = array([9, 7])
cayley_dickson(c1, d1, 3) = array([9, 7])


In [20]:
print(f"{qmult(q1, q2) = }")
print(f"{CayleyDickson(q1, q2) = }")
print(f"{cayley_dickson(q1, q2, 1) = }")
print(f"{cayley_dickson(q1, q2, 2) = }")
print(f"{cayley_dickson(q1, q2, 3) = }")

qmult(q1, q2) = array([35, -8, 24,  5])
CayleyDickson(q1, q2) = array([35., -8., 24.,  5.])
cayley_dickson(q1, q2, 1) = array([ 35,  22, -10,  -9])
cayley_dickson(q1, q2, 2) = array([35, -8, 24,  5])
cayley_dickson(q1, q2, 3) = array([-21,   6,  32,   3])
